Ensure pat2vec is on path

In [1]:
import sys
import os
import shutil
try:
    shutil.rmtree('new_project')
except Exception as e:
    print(e)

sys.path.insert(0,'/home/aliencat/samora/gloabl_files')
sys.path.insert(0,'/data/AS/Samora/gloabl_files')
sys.path.insert(0,'/home/jovyan/work/gloabl_files')
sys.path.insert(0,'/home/cogstack/samora/_data/gloabl_files')
sys.path.append('c:\\Users\\admin\\Documents\\projects\\pat2vec_time\\')
sys.path.append(r'C:\Users\admin\Documents\projects\gloabl_files')

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)

# Add the grandparent directory of the current directory to the Python path
grandparent_dir = os.path.dirname(parent_dir)
sys.path.append(grandparent_dir)


In [3]:
from util.config_pat2vec import config_class
from datetime import datetime
from tqdm import tqdm
from util.post_processing import process_csv_files
from util.post_processing import extract_datetime_to_column
from pat2vec_pat_list.get_patient_treatment_list import get_all_patients_list
from util.post_processing import produce_filtered_annotation_dataframe
from dateutil.relativedelta import relativedelta
import pandas as pd

# Configuration dictionary for main options in a medical application
main_options_dict = {
    # Enable demographic information (Ethnicity mapped to UK census, age, death)
    'demo': True,
    'bmi': True,  # Enable BMI (Body Mass Index) tracking
    'bloods': True,  # Enable blood-related information
    'drugs': False,  # Enable drug-related information
    'diagnostics': False,  # Enable diagnostic information

    'core_02': False,  # Enable core_02 information
    'bed': False,  # Enable bed n information
    'vte_status': False,  # Enable VTE () status tracking
    'hosp_site': False,  # Enable hospital site information
    'core_resus': False,  # Enable core resuscitation information
    'news': False,  # Enable NEWS (National Early Warning Score) tracking

    'smoking': False,  # Enable smoking-related information
    'annotations': True,  # Enable EPR annotations
    # Enable MRC (Additional clinical note observations index) annotations
    'annotations_mrc': True,
    # Enable or disable negated presence annotations
    'negated_presence_annotations': False
}


annot_filter_arguments = {
    'acc': 0.8,  # base concept accuracy
    # umls list of types for medcat filter
    'types': ['qualifier value', 'procedure', 'substance', 'finding', 'environment', 'disorder', 'observable entity'],
    # 'types': ['qualifier value', 'procedure', 'substance', 'finding', 'environment', 'disorder', 'observable entity', 'organism', 'phenomenon', 'anatomy', 'conceptual entity', 'physical object', 'intellectual product', 'occupation or discipline', 'mental or behavioral dysfunction', 'geographic area', 'population group', 'biomedical or dental material', 'medical device', 'classification', 'regulation or law', 'health care activity', 'health care related organization', 'professional or occupational group', 'group', 'attribute', 'individual behavior']
    # Specify the values you want to include in a list. Must be defined in medcat model.
    'Time_Value': ['Recent', 'Past'],
    'Time_Confidence': 0.8,  # Specify the confidence threshold as a float
    # Specify the values you want to include in a list
    'Presence_Value': ['True'],
    'Presence_Confidence': 0.8,  # Specify the confidence threshold as a float
    # Specify the values you want to include in a list
    'Subject_Value': ['Patient'],
    'Subject_Confidence': 0.8  # Specify the confidence threshold as a float
}

# Init config obj

# Creating a configuration object for a specific task or project
config_obj = config_class(
    remote_dump=False,  # Flag for remote data dumping. partially deprecated.
    suffix='',  # Suffix for file names
    # Filename for treatment documentation
    treatment_doc_filename='test_files/treatment_docs.csv',
    treatment_control_ratio_n=1,  # Ratio for treatment to control
    # Project name. patient data batches and vectors stored here.
    proj_name='new_project',
    current_path_dir="",  # Current path directory
    main_options=main_options_dict,  # Dictionary for main options
    start_date=(datetime(2020, 1, 1)),  # Starting date for processing
    # Number of years to add to the start date. Set the duration of the time window. Window is defined as the start date + years/months/days set here.
    years=0,
    months=0,  # Number of months to add to the start date
    days=2,  # Number of days to add to the start date
    # Flag for DGX, set true if in env, each env needs specific paths configured.
    dgx=False,
    dhcap=False,  # Flag for DHCap
    dhcap02=True,  # Flag for DHCap02
    batch_mode=True,  # Flag for batch processing mode. only functioning mode.
    store_annot=True,  # Flag to store annotations. partially deprecated.
    share_sftp=True,  # Flag for sharing via SFTP. partially deprecated
    multi_process=False,  # Flag for multi-process execution. deprecated.
    annot_first=False,  # Flag for annotation priority. deprecated.
    # Flag for stripping lists, will check for completed patients before starting to avoid redundancy.
    strip_list=True,
    verbosity=0,  # Verbosity level 0-9 printing debug messages
    random_seed_val=42,  # Random seed value for reproducibility of controls.
    testing=True,  # Flag for testing mode
    # Flag for using controls. #will add desired ratio of controls at random from global pool.
    use_controls=False,
    # Flag for MedCAT processing. #will load medcat into memory and use for annotating.
    medcat=True,
    # Current timestamp as the start time for logging and progress bar
    start_time=datetime.now(),
    # Column name for patient ID, auto will try to find it. Example "client_idcode"
    patient_id_column_name='auto',
    annot_filter_options=annot_filter_arguments,  # Annotation filtering options
    # Global start year. #set the limits of the time window data can be drawn from. Start should not precede start date set above.
    global_start_year=1995, # Global dates are overwritten by individual patient windows to match patient window.
    global_start_month=1,  # Global start month
    global_end_year=2023,  # Global end year
    global_end_month=1, # Global end month
    global_start_day = 1, 
    global_end_day = 1, 
    shuffle_pat_list=False,  # Flag for shuffling patient list
    time_window_interval_delta = relativedelta(years=1), #specify the time window to collapse each feature vector into, years=1 is one vector per year within the global time window
    split_clinical_notes=True, #will split clinical notes by date and treat as individual documents with extracted dates. Requires note splitter module. 
    lookback = True, # when calculating individual patient window from table of start dates, will calculate backwards in time if true. Else Forwards. 
    add_icd10 = False, #append icd 10 codes to annot batches
    add_opc4s=False, # needs icd10 true also.
    override_medcat_model_path = None, #Force medcat model path, if None uses defaults for env.
)

/home/cogstack/samora/_data/gloabl_files/pat2vec/notebooks/new_project/<bound method PathsClass._print_paths of <pat2vec.util.current_pat_batch_path_methods.PathsClass object at 0x7fccd446f6d0>>
/home/cogstack/samora/_data/gloabl_files/pat2vec/notebooks/new_project/['current_pat_annots_parts/', 'current_pat_annots_mrc_parts/', 'current_pat_documents_annotations_batches/', 'current_pat_documents_annotations_batches_mct/', 'current_pat_document_batches/', 'current_pat_document_batches_mct/', 'current_pat_bloods_batches/', 'current_pat_drugs_batches/', 'current_pat_diagnostics_batches/', 'current_pat_news_batches/', 'current_pat_obs_batches/', 'current_pat_bmi_batches/', 'current_pat_demo_batches/', 'current_pat_lines_parts/', 'outputs', '/home/cogstack/samora/_data/gloabl_files/pat2vec/notebooks/new_project/outputs']
/home/cogstack/samora/_data/gloabl_files/pat2vec/notebooks/new_project/
Setting start_date to: 2020-01-01 00:00:00
Setting years to: 0
Setting months to: 0
Setting days to: 

In [4]:
pwd

'/home/cogstack/samora/_data/gloabl_files/pat2vec/notebooks'

In [5]:
from main_pat2vec import main

refreshed
refreshed
******Watcher connected to ES Cluster!******

Cogstack toolbox functions:
cohort_searcher_with_terms_and_search(index_name, fields_list, term_name, entered_list, search_string) = Search with terms and search string
cohort_searcher_with_terms_no_search(index_name, fields_list, term_name, entered_list) = Search with terms only
cohort_searcher_no_terms(index_name, fields_list, search_string) = Search with search string only
matcher(data_template_df, lab_results_df, source_patid_colname, source_date_colname, result_date_colname, result_testname, result_resultname, before, after) = match template with dataset
**NOTE: matcher throws up an error if dates are not converted to datetime**
stringlist2searchlist(string_list, output_name) = convert a list of strings to a lucene search string
pylist2searchlist(list_name, output_name) = convert a list of strings to a python list
stringlist2pylist(string_list, var_name) = convert a python list to a lucene search string
date_cleaner

In [6]:
pat2vec_obj = main( cogstack=True, use_filter=False,
             json_filter_path = None, random_seed_val=42, 
             hostname =None, config_obj= config_obj, )


Init cohort_searcher_with_terms_and_search_dummy function
Initialized pat2vec.main
Setting NO gpu, most free memory: 3999 !


Bar desc:   0%|          | 0/3 [00:00<?, ?it/s]

Variable 'medcat_path' imported successfully from 'paths.py' file.
medcat_path: /home/cogstack/samora/_data/medcat_models/medcat_model_pack_bec3865f4a29ee20.zip


2024-02-06 22:08:16.780684: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-06 22:08:16.780739: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-06 22:08:16.782595: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-06 22:08:16.793206: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-06 22:08:17.935503: W tensorflow/comp

View patient list

In [7]:
pat2vec_obj.all_patient_list

['D3232DUM23', 'Z982DUM23', 'P545DUM23']

Make pat vectors for pat 0

In [8]:
pat2vec_obj.pat_maker(0)

s: None | D3232DUM23 | task: annot_pat_batch_docs_get_entities_multi_texts | {'n_docs_to_annotate': 4}:   0%|          | 0/3 [00:49<?, ?it/s]t/s]

2020 1 1 2023 1 1
2020 1 1 2023 1 1
2020 1 1 2023 1 1
2020 1 1 2023 1 1
2020 1 1 2023 1 1
generate_basic_observations_data


s: None | D3232DUM23_(2020, 1, 1) | task: Columns n=8 | {}:   0%|          | 0/3 [00:50<?, ?it/s]t/s]                                            

In [9]:
# Remove specific patient raw documents and annotations:
#from pat2vec.util.post_processing import remove_file_from_paths

# remove_file_from_paths(pat2vec_obj.all_patient_list[i])

In [10]:


for i in tqdm(range(0,len(pat2vec_obj.all_patient_list))):
    
    pat2vec_obj.pat_maker(i)
    pat2vec_obj.t.update(1)
    
pat2vec_obj.t.close()

s: None | Z982DUM23 | task: annot_pat_batch_docs_get_entities_multi_texts | {'n_docs_to_annotate': 9}:  33%|███▎      | 1/3 [00:50<01:40, 50.11s/it]/it]

2020 1 1 2023 1 1
2020 1 1 2023 1 1
2020 1 1 2023 1 1
generate_basic_observations_data
2020 1 1 2023 1 1
2020 1 1 2023 1 1
2020 1 1 2023 1 1
2020 1 1 2023 1 1
2020 1 1 2023 1 1
generate_basic_observations_data


s: None | P545DUM23 | task: annot_pat_batch_docs_get_entities_multi_texts | {'n_docs_to_annotate': 9}:  67%|██████▋   | 2/3 [00:50<00:21, 21.05s/it]/it]

2020 1 1 2023 1 1
2020 1 1 2023 1 1
2020 1 1 2023 1 1
2020 1 1 2023 1 1
2020 1 1 2023 1 1
generate_basic_observations_data


100%|██████████| 3/3 [00:01<00:00,  1.79it/s]mColumns n=8 | {}: 100%|██████████| 3/3 [00:51<00:00, 11.86s/it]/it]                                            
s: None | P545DUM23_(2020, 1, 1) | task: Columns n=8 | {}: 100%|██████████| 3/3 [00:51<00:00, 17.25s/it]


In [11]:
import os
import polars as pl
import pandas as pd

from util.post_processing import process_csv_files_multi

# Set the directory paths and file names
input_directory = f'{pat2vec_obj.proj_name}/current_pat_lines_parts'
output_csv_file = 'concatenated_output'
out_folder = f'{pat2vec_obj.proj_name}/outputs'
out_file_path = os.path.join(out_folder, f'concatenated_data_{output_csv_file}.csv')
overwrite = True
use_polars = False

# Check if the output file exists and if overwrite is set to False
if os.path.exists(out_file_path) and not overwrite:
    print(f'The file {out_file_path} exists and overwrite is set to False')
    if use_polars:
        dfp = pl.read_csv(out_file_path)
        df = dfp.to_pandas()
    else:
        df = pd.read_csv(out_file_path, engine='pyarrow')
else:
    print(f'The file {out_file_path} does not exist or overwrite is set to True')
    try:
        # Call the function to process CSV files
        process_csv_files_multi(input_directory, out_folder=out_folder, output_filename_suffix=output_csv_file, part_size=20, sample_size=None, n_proc='half')
    except Exception as e:
        print(e)
    
    # Read the output file after processing
    if use_polars:
        dfp = pl.read_csv(out_file_path)
        df = dfp.to_pandas()
    else:
        df = pd.read_csv(out_file_path, engine='pyarrow')


The file new_project/outputs/concatenated_data_concatenated_output.csv does not exist or overwrite is set to True
all files size 3


100%|██████████| 3/3 [00:00<00:00, 3083.29it/s]

desried cores: 24



1it [00:00, 455.80it/s]
Writing lines...: 100%|██████████| 1/1 [00:00<00:00, 9892.23it/s]

Concatenated data saved to new_project/outputs/concatenated_data_concatenated_output.csv


In [12]:
df = pd.read_csv(out_file_path)

In [13]:
df

,client_idcode,Unnamed: 1,"(2020, 1, 1)_date_time_stamp",order_entered,order_name,order_holdreasontext,clientvisit_visitidcode,order_summaryline,order_guid
0,D3232DUM23,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,Z982DUM23,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,P545DUM23,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:



df = extract_datetime_to_column(df)

100%|██████████| 3/3 [00:00<00:00, 1353.87it/s]

extracted_datetime_stamp
2020-01-01    3
Name: count, dtype: int64


### Filter the annotation batches by a snomed cui and its related codes. 

In [15]:
from snomed_methods import snomed_methods_v1
snomed_relations_obj = snomed_methods_v1.snomed_relations(medcat=True, snomed_rf2_full_path=None)

outcome_variable_cui_for_filter = '40733004'  # infection

print(outcome_variable_cui_for_filter)

filter_root_cui = outcome_variable_cui_for_filter
print(filter_root_cui)

retrieved_codes_snomed_tree, retrieved_names_snomed_tree = snomed_relations_obj.recursive_code_expansion(filter_root_cui, n_recursion = 3, debug=False)

retrieved_codes_snomed_tree[0:5], len(retrieved_codes_snomed_tree), len(retrieved_names_snomed_tree)





40733004
40733004
Retrieving 40733004 with recursion 3


100%|██████████| 3/3 [00:05<00:00,  1.80s/it]


([20480004, 27648007, 46080009, 42197002, 40149008], 1690, 1395)

In [16]:
retrieved_names_snomed_tree[0:10]

['Tonsillar actinomycosis (disorder)',
 'Abrasion and/or friction burn of ankle with infection (disorder)',
 'Disease caused by Gram-negative bacillus (disorder)',
 'Rhinosporidial papilloma (disorder)',
 'Nonvenomous insect bite of wrist with infection (disorder)',
 'Abscess gonococcal (disorder)',
 'Hepatitis due to infection (disorder)',
 'Malarial pigment deposition (disorder)',
 'Emphysematous cystitis (disorder)',
 'Methicillin resistant Staphylococcus aureus infection (disorder)']

In [17]:
retrieved_codes_medcat_cdb, retrieved_names_medcat_cdb  = snomed_relations_obj.get_medcat_cdb_most_similar(filter_root_cui, context_type = 'xxxlong', type_id_filter=[], topn=25)

In [18]:
retrieved_names_medcat_cdb[0:10]

['Infectious disease (disorder)',
 'Bacterial infectious disease (disorder)',
 'Acute infectious disease (disorder)',
 'Superimposed infection (disorder)',
 'Localized infection (disorder)',
 'Viral disease (disorder)',
 'Mycoplasma infection (disorder)',
 'Source of infection (attribute)',
 'Anemia due to infection (disorder)',
 'New infection (qualifier value)']

In [19]:
all_names_list = list(set(retrieved_names_medcat_cdb + retrieved_names_snomed_tree))

all_codes_list = list(set(retrieved_codes_medcat_cdb + retrieved_codes_snomed_tree))

print(len(all_names_list))

1405


In [20]:


all_pat_list_ = get_all_patients_list(config_obj=pat2vec_obj.config_obj)


all_annot_filtered_df = produce_filtered_annotation_dataframe(cui_filter=True, meta_annot_filter=True, pat_list=all_pat_list_, config_obj=pat2vec_obj.config_obj, filter_custom_args=pat2vec_obj.config_obj.annot_filter_options, cui_code_list=all_codes_list)

100%|██████████| 3/3 [00:00<00:00, 156.32it/s]


In [21]:
all_annot_filtered_df

,Unnamed: 0,client_idcode,updatetime,pretty_name,cui,type_ids,types,source_value,detected_name,acc,...,id,Time_Value,Time_Confidence,Presence_Value,Presence_Confidence,Subject_Value,Subject_Confidence,text_sample,full_doc,document_guid


In [24]:
from patvec_get_batch_methods.main import get_pat_batch_epr_docs


current_pat_client_id_code = ''

config_obj = config_obj

cohort_searcher_with_terms_and_search = pat2vec_obj.cohort_searcher_with_terms_and_search

get_pat_batch_epr_docs(current_pat_client_id_code, 'search_term', config_obj=config_obj, cohort_searcher_with_terms_and_search=cohort_searcher_with_terms_and_search)
  

2020 1 1 2023 1 1
Error retrieving batch EPR documents: Cannot save file into a non-existent directory: '/home/cogstack/samora/_data/gloabl_files/pat2vec/notebooks/new_project/current_pat_document_batches'


UnboundLocalError: Error retrieving batch EPR documents.

In [ ]:

df = generate_epr_documents_data(num_rows, entered_list, global_start_year, global_start_month, global_end_year, global_end_month)